<img src="https://dutchanalytics.com/wp-content/uploads/2020/02/svg_logo_dutch_analytics-1.svg" width="80">

# Quickstart MNIST demo
Deploy your deployment in UbiOps using the python client library.


The first step is to download a prepared deployment and some sample data.

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/mnist_deployment_package.zip -o mnist_deployment_package.zip
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/1.jpg -o 1.jpg
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/2.jpg -o 2.jpg
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/3.jpg -o 3.jpg

In [ ]:
image_files = ['1.jpg', '2.jpg', '3.jpg']

Add your API token, provide a project name, deployment name and deployment version name.

In [ ]:
API_TOKEN = 'Token <YOUR_API_KEY>'
PROJECT_NAME = '<YOUR_PROJECT_NAME>'
DEPLOYMENT_NAME = 'mnist-tutorial'
DEPLOYMENT_VERSION = 'v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration()
configuration.api_key['Authorization'] = API_TOKEN

Here we open the connection with the UbiOps API Client.

In [ ]:
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy
Create a deploytment for mnist.

In [ ]:
mnist_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='A deployment to classify handwritten digits.',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.DeploymentInputFieldCreate(name='image', data_type='blob')
    ],
    output_fields=[
        ubiops.DeploymentOutputFieldCreate(name='prediction', data_type='int'),
        ubiops.DeploymentOutputFieldCreate(name='probability', data_type='double')
    ]
)

mnist_deployment = api.deployments_create(project_name=PROJECT_NAME, data=mnist_template)
print(mnist_deployment)

Create a deployment version for mnist.

In [ ]:
version_template = ubiops.VersionCreate(
    version=DEPLOYMENT_VERSION,
    language='python3.6',
    memory_allocation=1024,
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800 # = 30 minutes
)

version = api.versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)
print(version)

Upload the prepared deployment file to the created version.

In [ ]:
upload_response = api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='mnist_deployment_package.zip'
)
print(upload_response)

Check if deployment is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'queued'
while status != 'success' and status != 'failed':    
    build_status = api.builds_get(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        version=DEPLOYMENT_VERSION,
        build_id=upload_response.build
    )
    status = build_status.status
    print("{:15s}".format(status), end='\r')
    sleep(1)
print(status)

The version is now available.

In [ ]:
api.versions_get(       
    project_name=PROJECT_NAME,        
    deployment_name=DEPLOYMENT_NAME,        
    version=DEPLOYMENT_VERSION    
).status

## Create multiple direct requests
Make sure the deployment is in 'available' state before performing deployment requests.

Create a deployment request for each image and get the results directly if the request is finished.

The first request is usually slow because of a cold start. A second request (performed within the `maximum_idle_time`, which was set to 30 minutes in this tutorial) will be much faster.

In [ ]:
from IPython.display import Image, display

for image_file in image_files:
    # Display the input image
    display(Image(url=image_file, width=40, height=40))
    
    # First upload the image
    blob = api.blobs_create(project_name=PROJECT_NAME, file=image_file)
    
    # Make a request using the blob id as input.
    data = {'image': blob.id}
    request_result = api.deployment_requests_create(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        version=DEPLOYMENT_VERSION,
        data=data
    )
    print(request_result)

## Create single batch request
Make sure the deployment is in 'available' state before performing deployment requests. <br/>
Create a deployment batch request for all images at ones. This will create multiple asynchronuous requests, which means that the requests are queued in the back-end and the results can be collected at a later time.

In [ ]:
from IPython.display import Image, display

# Upload the images
blobs = []
for image_file in image_files:
    blobs.append(api.blobs_create(project_name=PROJECT_NAME, file=image_file))
    
# Make a batch request using the blob ids as input.
data = [{'image': blob.id} for blob in blobs]
created_batch = api.batch_deployment_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    data=data
)

# Display
for image_file, request in zip(image_files, created_batch):
    display(Image(url=image_file, width=40, height=40))
    print(request)

## Wait for results
Display request results when they are finished.

In [ ]:
from time import sleep

for request in created_batch:
    print(request.id, '', end='\r')
    
    status = 'pending'
    while status != 'completed' and 'failed' not in status:    
        request_response = api.batch_deployment_requests_get(       
            project_name=PROJECT_NAME,        
            deployment_name=DEPLOYMENT_NAME,        
            version=DEPLOYMENT_VERSION,
            request_id=request.id
        )    
        status = request_response.status
        sleep(1)
    print(request.id, status, request_response.result)

## List all batch requests
List all batch requests of the deployment. <br/>
Note that batch requests are stored for a limited time. See your subscription for more information.

In [ ]:
import pprint

all_batch_requests = api.batch_deployment_requests_list(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
)
pprint.pprint(all_batch_requests)

## Cleanup
Delete created deployment.

In [ ]:
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME
)

## Close connection
Close the connection with the UbiOps API client.

In [ ]:
client.close()